In [6]:
import re
import pandas as pd
import time
import json
from groq import Groq

In [ ]:
# init groq client
client = Groq(
    api_key='<paste_groq_key_here>',
)

In [14]:
# read settings
prompts = pd.read_csv('data/prompts.csv')

# read articles
articles = pd.read_csv('data/evaluation_texts.csv')

# read criteria
criteria = pd.read_csv('data/criteria.csv')
criteria = criteria['kriterien'].tolist()

# define techniques
techniques = prompts['key'].tolist()

In [4]:
max_len = 15000 # max char length of an article (to save tokens due to the groq tokens / minutes limit)
model = "llama3-8b-8192"

In [ ]:
all_results = []

# do three runs 
for run in [0, 1, 2]:

    print('\n#### - RUN:', run+1, " - ####\n")

    # init list for run    
    results_per_run = []
    
    # iterate over all techniques
    for technik in techniques:

        print('\nNEXT PROMPT STYLE:', technik, '\n')

        # get criteria right
        krits = [f'{i+1} {criteria[i]}' for i in range(len(criteria))] if "each" in technik else '\n'.join(f'{i+1} {criteria[i]}' for i in range(len(criteria)))
        if len(krits) != 10: krits = [krits]

        # get instruction right
        instruction = prompts[prompts['key'] == technik]['value'].tolist()

        # init predictions_list that entails all articles
        predictions = []

        # iterate over all documents
        for row in articles.itertuples():

            print('  article:', row.id)
            
            # strip text if needed
            text = row.text if len(row.text) < max_len else row.text[:max_len]
            
            # init prediction_per_crit list that entails the predictions for one single document
            prediction_per_crit = []

            # iterate over all criteria
            for krit in krits:

                print('   crit:', krit)

                # create complete promt
                prompt = f'{instruction}\n\n#ARTIKEL:\n{text}\n\n#KRIT.:\n{krit}'

                try: # for time and token limits when using groq
                    chat_completion = client.chat.completions.create(
                        messages=[
                            {"role": "system", "content": "Du bist strenger wissenschaftlicher Gutachter von wissenschaftsjournalistischen Texten."},
                            {"role": "user", "content": prompt},
                        ],
                        temperature=0.1,
                        model=model,
                    )
                except: # for time and token limits when using groq
                    print('take a nap...')
                    time.sleep(120)
                    print('try again...')
                    chat_completion = client.chat.completions.create(
                        messages=[
                            {"role": "system", "content": "Du bist strenger wissenschaftlicher Gutachter von wissenschaftsjournalistischen Texten."},
                            {"role": "user", "content": prompt},
                        ],
                        temperature=0.1,
                        model=model,
                    )

                # append predictions_per_crit list (one article - all predictions)
                prediction_per_crit.append(chat_completion.choices[0].message.content)

            # append predictions list (all articles - same prompting)
            predictions.append(prediction_per_crit)

        # append results_per_run list all prompt - one out of three runs
        results_per_run.append(predictions)

    # append all_results list - all data
    all_results.append(results_per_run)

# return
all_results

In [ ]:
# save locally
with open(model+"evaluations.json", "w", encoding="utf-8") as outfile: 
    json.dump(all_results, outfile, indent=4, ensure_ascii=False)